In [52]:
# Imports
import geopandas as gpd
import pandas as pd
import dbfread as dbf

## Part 2

In [55]:
# Reading in shapefile data
saep_bg10 = gpd.read_file("saep_bg10/saep_bg10.shp")
saep_bg10.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,FUNCSTAT10,INTPTLON10,INTPTLAT10,...,OHU2014,OHU2015,OHU2016,OHU2017,OHU2018,COHU00_10,PCOHU00_10,COHU10_18,PCOHU10_18,geometry
0,53,001,950100,1,530019501001,Block Group 1,G5030,S,-118.398815,47.150809,...,277.494,279.680,287.530,287.522,287.758,33.751,0.143,17.758,0.066,"POLYGON ((2077217.074095237 640954.0110817049,..."
1,53,001,950100,2,530019501002,Block Group 2,G5030,S,-118.351143,47.135101,...,524.831,521.228,517.427,516.077,524.645,3.398,0.007,-0.355,0.000,"POLYGON ((2165913.051240579 657352.8005365322,..."
2,53,001,950100,3,530019501003,Block Group 3,G5030,S,-118.382148,47.138222,...,246.344,243.965,242.033,241.365,245.491,-26.312,-0.097,-0.509,-0.002,"POLYGON ((2166254.454566129 657810.7137267586,..."
3,53,001,950200,1,530019502001,Block Group 1,G5030,S,-118.219484,46.874027,...,208.923,203.761,205.502,204.547,202.854,-21.255,-0.093,-5.146,-0.025,"POLYGON ((2150159.78406948 554295.8999707697, ..."
4,53,001,950200,2,530019502002,Block Group 2,G5030,S,-118.522323,46.968569,...,285.857,286.251,278.398,278.400,267.983,-1.330,-0.005,-22.017,-0.076,"POLYGON ((2082999.834788324 575018.8611698836,..."


In [56]:
# Reading in DBF data and converting into a Panda's dataframe
WashingtonFIPS = dbf.DBF('WashingtonFIPS.dbf')
fips_df = pd.DataFrame(iter(WashingtonFIPS))
fips_df.head()

,CountyName,FIPSCounty
0,Adams,001
1,Asotin,003
2,Benton,005
3,Chelan,007
4,Clallam,009


In [70]:
# Creating shapefiles and gathering population sums for each county

# Creating empty dataframe to store population sums 
counties = pd.DataFrame(columns=['county_name', 'county_population'])

# Iterate through the fips_df to get individual county name and number
for index, row in fips_df.iterrows():
    fips = row.FIPSCounty
    name = row.CountyName
    
    # Using the fips from fips_df
    # Select using loc to select the rows of saep_bg10 where the 
    # COUNTYFP10 == fips of current county
    # i.e. selects the rows from saep_bg10 for each county
    polygons = saep_bg10.loc[saep_bg10['COUNTYFP10'] == fips]
    
    # Adding the county name and sum of the populations for 2017 from 
    # the polygons made above
    counties.loc[index] = [name, sum(polygons['POP2017'])]
    
    # Creates the filename and adds the polygons for each county to the 
    # shapefile for that county
    filename = "counties/" + name + ".shp"
#     polygons.to_file(filename)
    
# Sorting the counties by the population
counties.sort_values(by=['county_population'], ascending=False).head()

,county_name,county_population
16,King,2153700.0
26,Pierce,859400.0
30,Snohomish,789400.0
31,Spokane,499800.0
5,Clark,471000.0
